In [1]:
import os
os.getcwd()

'c:\\Users\\somit\\Downloads\\project_ineuron\\insurence_premium_prediction\\research'

In [2]:
os.chdir("..")
os.getcwd()

'c:\\Users\\somit\\Downloads\\project_ineuron\\insurence_premium_prediction'

In [3]:
from collections import namedtuple

In [175]:
from pathlib import Path
from dataclasses import dataclass


@dataclass(frozen=True)
class DataModelTrainerConfig:
    root_dir: Path
    trained_model_file_path: Path
    base_accuracy: float
    model_config_file_path: Path


In [177]:
read_yaml()

In [178]:
from insurence_premium.constant import *
from insurence_premium.util.common import (
    read_yaml,
    create_directories,
    load_numpy_array_data,
)
from pathlib import Path
from insurence_premium.entity import (
    DataIngestionConfig,
    DataValidationConfig,
    DataTransformationConfig,
)
from insurence_premium import logger


class ConfigurationManager:
    def __init__(
        self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH
    ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(
        self,
    ) -> DataIngestionConfig:
        ingestion_config = self.config.data_ingestion
        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(ingestion_config.root_dir),
            raw_data=Path(ingestion_config.raw_data),
            ingested_train_dir=Path(ingestion_config.ingested_train_dir),
            ingested_test_dir=Path(ingestion_config.ingested_test_dir),
        )
        return data_ingestion_config

    def get_data_validation_config(self) -> DataValidationConfig:
        validation_config = self.config.data_validation

        data_validation_config = DataValidationConfig(
            root_dir=Path(validation_config.root_dir),
            schema_file_path=Path(validation_config.schema_file_path),
            report_file_path=Path(validation_config.report_file_path),
            report_page_file_path=Path(validation_config.report_page_file_path),
        )
        return data_validation_config

    def get_data_transformation_config(self) -> DataTransformationConfig:
        try:
            transformation_config = self.config.data_transformation
            data_transformation_config = DataTransformationConfig(
                root_dir=Path(transformation_config.root_dir),
                transformed_train_file_path=Path(
                    transformation_config.transformed_train_file_path
                ),
                transformed_test_file_path=Path(
                    transformation_config.transformed_test_file_path
                ),
                preprocessing_dir=Path(transformation_config.preprocessing_dir),
                preprocessing_file_path=Path(
                    transformation_config.preprocessing_file_path
                ),
            )
            return data_transformation_config
            logging.info(f"return: [{data_ingestion_config}]")
        except Exception as e:
            raise e

    def get_model_trainer_config(self) -> DataModelTrainerConfig:
        try:
            model_trainer_config = self.config.data_model_trainer

            data_model_trainer_config = DataModelTrainerConfig(
                root_dir=Path(model_trainer_config.root_dir),
                trained_model_file_path=Path(
                    model_trainer_config.trained_model_file_path
                ),
                base_accuracy=model_trainer_config.base_accuracy,
                model_config_file_path=Path(
                    model_trainer_config.model_config_file_path
                ),
            )
            return data_model_trainer_config
            logger.info(f"model trainer config : {data_model_trainer_config}")
        except Exception as e:
            raise e

    def get_model_pusher_config(self):
        try:
            pass
        except Exception as e:
            raise e


In [179]:
read_yaml(path_to_yaml=Path("config/config.yaml"))

[2023-03-30 14:21:23,109: INFO: common]: yaml file: config\config.yaml loaded successfully


ConfigBox({'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'raw_data': 'artifacts/data_ingestion/raw_data/insurence.csv', 'ingested_train_dir': 'artifacts/data_ingestion/train/insurence.csv', 'ingested_test_dir': 'artifacts/data_ingestion/test/insurence.csv'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'report_file_path': 'artifacts/data_validation/report.json', 'report_page_file_path': 'artifacts/data_validation/report.html', 'schema_file_path': 'config/schema.yaml'}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'transformed_train_file_path': 'artifacts/data_transformation/train/insurence.npz', 'transformed_test_file_path': 'artifacts/data_transformation/test/insurence.npz', 'preprocessing_dir': 'artifacts/data_transformation/preprocessed_file', 'preprocessing_file_path': 'artifacts/data_transformation/preprocessed_file/preprocessing.pkl'}, 'data_model_trainer': {'root_dir': 'artifacts/data_model_train

In [180]:
ConfigurationManager().get_model_trainer_config()

[2023-03-30 14:21:23,897: INFO: common]: yaml file: config\config.yaml loaded successfully
[2023-03-30 14:21:23,903: INFO: common]: yaml file: params.yaml loaded successfully
[2023-03-30 14:21:23,907: INFO: common]: created directory at: artifacts


DataModelTrainerConfig(root_dir=WindowsPath('artifacts/data_model_trainer'), trained_model_file_path=WindowsPath('artifacts/data_model_trainer/trained_model/model.pkl'), base_accuracy=0.6, model_config_file_path=WindowsPath('config/model.yaml'))

In [181]:
con =ConfigurationManager()
data_trainer_config=con.get_model_trainer_config()
model_config_file_path=data_trainer_config.model_config_file_path
m_i_c:dict=dict(ModelFactory.read_params(model_config_file_path)[GRID_SEARCH_KEY][PARAM_KEY])
m_i_c

[2023-03-30 14:21:24,375: INFO: common]: yaml file: config\config.yaml loaded successfully
[2023-03-30 14:21:24,379: INFO: common]: yaml file: params.yaml loaded successfully
[2023-03-30 14:21:24,384: INFO: common]: created directory at: artifacts


{'cv': 5, 'verbose': 2}

In [196]:
class HousingEstimatorModel:
    def __init__(self, preprocessing_object, trained_model_object):
        """
        TrainedModel constructor
        preprocessing_object: preprocessing_object
        trained_model_object: trained_model_object
        """
        self.preprocessing_object = preprocessing_object
        self.trained_model_object = trained_model_object

    def predict(self, X):
        """
        function accepts raw inputs and then transformed raw input using preprocessing_object
        which gurantees that the inputs are in the same format as the training data
        At last it perform prediction on transformed features
        """
        transformed_feature = self.preprocessing_object.transform(X)
        return self.trained_model_object.predict(transformed_feature)

    def __repr__(self):
        return f"{type(self.trained_model_object).__name__}()"

    def __str__(self):
        return f"{type(self.trained_model_object).__name__}()"



class ModelTrainer:
    def __init__(self, config: ConfigurationManager):
        self.config = ConfigurationManager()
        self.data_trainer_config = self.config.get_model_trainer_config()
        self.data_transformation_config = self.config.get_data_transformation_config()

    def get_trained_model(self):
        try:
            logger.info("loading training and testing data sets")
            train_file_path = self.data_transformation_config.transformed_train_file_path
            test_file_path = self.data_transformation_config.transformed_test_file_path

            train_data = load_numpy_array_data(file_path=train_file_path)
            test_data = load_numpy_array_data(file_path=test_file_path)
            x_train, y_train = train_data[:, :-1], train_data[:, -1]
            x_test, y_test = test_data[:, :-1], test_data[:, -1]
            logger.info("train and test data generated for data training")

            model_config_file_path = self.data_trainer_config.model_config_file_path
            logger.info("initializing model by using config filepath")

            model_factory = ModelFactory(config_file_path=model_config_file_path)

            base_accuracy = self.data_trainer_config.base_accuracy
            logger.info(f"expected accuracy:{base_accuracy}")

            best_model = model_factory.get_best_model(x=x_train,y=y_train,base_accuracy=base_accuracy)
            
            logger.info(f"Best model found on training dataset: {best_model}")
            
            logger.info(f"Extracting trained model list.")
            grid_searched_best_model_list:List[GridSearchedBestModel]=model_factory.grid_searched_best_model_list

            model_list = [model.best_model for model in grid_searched_best_model_list ]
            logger.info(f"Evaluation all trained model on training and testing dataset both")
            metric_info:MetricInfoArtifact = evaluate_regression_model(model_list=model_list,x_train=x_train,y_train=y_train,x_test=x_test,y_test=y_test,base_accuracy=base_accuracy)

            logger.info(f"Best found model on both training and testing dataset.")
            preprocessing_obj=  load_object(file_path=self.data_transformation_config.preprocessing_file_path)
            model_object = metric_info.model_object


            trained_model_file_path=self.data_trainer_config.trained_model_file_path
            housing_model = HousingEstimatorModel(preprocessing_object=preprocessing_obj,trained_model_object=model_object)
            logger.info(f"Saving model at path: {trained_model_file_path}")
            save_object(file_path=trained_model_file_path,obj=housing_model)
            logger.info(f"Saving model at path: {trained_model_file_path} is successfully done")




        except Exception as e:
            raise e




In [197]:
GRID_SEARCH_KEY = "grid_search"
MODULE_KEY = "module"
CLASS_KEY = "class"
PARAM_KEY = "params"
MODEL_SELECTION_KEY = "model_selection"
SEARCH_PARAM_GRID_KEY = "search_param_grid"


In [198]:
InitialisedModelDetail = namedtuple(
    "InitialisedModelDetail",
    ["model_serial_number", "model", "param_grid_search", "model_name"],
)

GridSearchedBestModel = namedtuple(
    "GridSearchedBestModel",
    [
        "model_serial_number",
        "model",
        "best_model",
        "best_parameters",
        "best_score",
    ],
)

BestModel = namedtuple(
    "BestModel",
    [
        "model_serial_number",
        "model",
        "best_model",
        "best_parameters",
        "best_score",
    ],
)

MetricInfoArtifact = namedtuple(
    "MetricInfoArtifact",
    [
        "model_name",
        "model_object",
        "train_rmse",
        "test_rmse",
        "train_accuracy",
        "test_accuracy",
        "model_accuracy",
        "index_number",
    ],
)


In [199]:
class ModelFactory:
    def __init__(self ,config_file_path:str):
        self.config=ModelFactory.read_params(config_file_path)
        self.grid_search_cv_module:str = self.config[GRID_SEARCH_KEY][MODULE_KEY]
        self.grid_search_cv_class:str = self.config[GRID_SEARCH_KEY][CLASS_KEY]
        self.grid_search_cv_property_data:dict = dict(self.config[GRID_SEARCH_KEY][PARAM_KEY])
        self.models_initialization_config: dict = dict(self.config[MODEL_SELECTION_KEY])
        self.initialised_model_list = None
        self.grid_searched_best_model_list = None

    @staticmethod
    def read_params(file_path:str):
        try:
            with open(file_path) as file_obj:
                config:dict =yaml.safe_load(file_obj)
            return config    

        except Exception as e:
            raise e   

    @staticmethod
    def update_property_for_class(instance_obj:object ,property_data:dict):
        try:
            if not isinstance(property_data ,dict):
                raise Exception("property_data required dict formate")
                print(proprty_data)
            for key ,value in property_data.items():
                setattr(instance_obj ,key ,value)
                logger.info(f"Executing:$ {str(instance_obj)}.{key}={value}")
            return instance_obj
        except Exception as e:
            raise e        

    @staticmethod
    def class_for_name(module_name:str ,class_name=str):
        try:
            module=importlib.import_module(module_name)
            logger.info(f"importing module:{module_name} with class:{class_name}")
            class_ref=getattr(module ,class_name)
            return class_ref

        except Exception as e:
            raise e

    def get_initialised_model_list(self)->List[InitialisedModelDetail]:
        try:
            initialised_model_list=[]
            for model_serial_number in self.models_initialization_config.keys():
                model_initialization_config =self.models_initialization_config[model_serial_number]
                model_obj_ref =ModelFactory.class_for_name(
                    model_initialization_config[MODULE_KEY] ,
                    model_initialization_config[CLASS_KEY])
                model=model_obj_ref()

                if PARAM_KEY in model_initialization_config:
                    model_property_data=dict(model_initialization_config[PARAM_KEY]) 
                    model=ModelFactory.update_property_for_class(instance_obj=model ,property_data=model_property_data)
                param_grid_search=model_initialization_config[SEARCH_PARAM_GRID_KEY]

                model_name=f"[{model_initialization_config[MODULE_KEY]}.{model_initialization_config[CLASS_KEY]}]"

                model_initialization_detail=InitialisedModelDetail(
                    model_serial_number=model_serial_number,
                    model=model,
                    param_grid_search=param_grid_search,
                    model_name=model_name,
                    
                )   

                initialised_model_list.append(model_initialization_detail)
            return initialised_model_list        
        except Exception as e:
            raise e
    
    def execute_grid_search_operation(self,

                                    initialised_model:InitialisedModelDetail ,
                                    input_feature ,output_feature)->GridSearchedBestModel:
        try:
            grid_search_cv_ref=ModelFactory.class_for_name(module_name=self.grid_search_cv_module ,class_name=self.grid_search_cv_class)
            grid_search_cv=grid_search_cv_ref(estimator=initialised_model.model ,param_grid=initialised_model.param_grid_search)
            grid_search_cv=ModelFactory.update_property_for_class(instance_obj=grid_search_cv ,property_data=self.grid_search_cv_property_data)
            grid_search_cv.fit(input_feature ,output_feature)

            message = f'{">>"* 30} f"Training {type(initialised_model.model).__name__}" completed {"<<"*30}'
            grid_searched_best_model = GridSearchedBestModel(model_serial_number=initialised_model.model_serial_number,
                                                             model=initialised_model.model,
                                                             best_model=grid_search_cv.best_estimator_,
                                                             best_parameters=grid_search_cv.best_params_,
                                                             best_score=grid_search_cv.best_score_
                                                             )
            
            return grid_searched_best_model
        
        except Exception as e:
            raise e                            
    
    def initiate_best_parameter_search_for_initialised_model(self, initialised_model: InitialisedModelDetail,
                                                             input_feature,
                                                             output_feature) -> GridSearchedBestModel:
        """
        initiate_best_model_parameter_search(): function will perform paramter search operation and
        it will return you the best optimistic  model with best paramter:
        estimator: Model object
        param_grid: dictionary of paramter to perform search operation
        input_feature: your all input features
        output_feature: Target/Dependent features
        ================================================================================
        return: Function will return a GridSearchOperation
        """
        try:
            return self.execute_grid_search_operation(initialised_model=initialised_model,
                                                      input_feature=input_feature,
                                                      output_feature=output_feature)
        except Exception as e:
            raise e
    def initiate_best_parameter_search_for_initialised_models(self,
                                                              initialised_model_list: List[InitialisedModelDetail],
                                                              input_feature,
                                                              output_feature) -> List[GridSearchedBestModel]:

        try:
            self.grid_searched_best_model_list = []
            for initialised_model_list in initialised_model_list:
                grid_searched_best_model = self.initiate_best_parameter_search_for_initialised_model(
                    initialised_model=initialised_model_list,
                    input_feature=input_feature,
                    output_feature=output_feature
                )
                self.grid_searched_best_model_list.append(grid_searched_best_model)
            return self.grid_searched_best_model_list
        except Exception as e:
            raise e
    @staticmethod
    def get_best_model_from_grid_searched_best_model_list(grid_searched_best_model_list:List[GridSearchedBestModel] ,base_accuracy=0.6)->BestModel:
        try:
            best_model=None
            for grid_search_best_model in grid_searched_best_model_list:
                if grid_search_best_model.best_score > base_accuracy:
                    logger.info("Acceptable model found:{grid_search_best_model}")
                    best_model=grid_search_best_model


            if not best_model:
                raise Exception("No better model has accuracy than base model")  
            logger.info(f"best model found:{best_model}")     
        except Exception as e:
            raise e                

    def get_best_model(self ,x,y,base_accuracy=0.6):
        try:
            initialised_model=self.get_initialised_model_list()
            grid_searched_best_model_list = self.initiate_best_parameter_search_for_initialised_models(
                initialised_model_list=initialised_model,
                input_feature=x,
                output_feature=y
            )
            return ModelFactory.get_best_model_from_grid_searched_best_model_list(grid_searched_best_model_list=grid_searched_best_model_list,base_accuracy=0.6)
        except Exception as e:
            raise e    

In [200]:
GRID_SEARCH_KEY = 'grid_search'
MODULE_KEY = 'module'
CLASS_KEY = 'class'
PARAM_KEY = 'params'
MODEL_SELECTION_KEY = 'model_selection'
SEARCH_PARAM_GRID_KEY = "search_param_grid"


def evaluate_regression_model(model_list: list, x_train:np.ndarray, y_train:np.ndarray, x_test:np.ndarray, y_test:np.ndarray, base_accuracy:float=0.6) -> MetricInfoArtifact:
    """
    Description:
    This function compare multiple regression model return best model
    Params:
    model_list: List of model
    X_train: Training dataset input feature
    y_train: Training dataset target feature
    X_test: Testing dataset input feature
    y_test: Testing dataset input feature
    return
    It retured a named tuple
    
    MetricInfoArtifact = namedtuple("MetricInfo",
                                ["model_name", "model_object", "train_rmse", "test_rmse", "train_accuracy",
                                 "test_accuracy", "model_accuracy", "index_number"])
    """
    try:
        
    
        index_number = 0
        metric_info_artifact = None
        for model in model_list:
            model_name = str(model)  #getting model name based on model object
            logger.info(f"{'>>'*30}Started evaluating model: [{type(model).__name__}] {'<<'*30}")
            
            #Getting prediction for training and testing dataset
            y_train_pred = model.predict(x_train)
            y_test_pred = model.predict(x_test)

            #Calculating r squared score on training and testing dataset
            train_acc = r2_score(y_train, y_train_pred)
            test_acc = r2_score(y_test, y_test_pred)
            
            #Calculating mean squared error on training and testing dataset
            train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
            test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

            # Calculating harmonic mean of train_accuracy and test_accuracy
            model_accuracy = (2 * (train_acc * test_acc)) / (train_acc + test_acc)
            diff_test_train_acc = abs(test_acc - train_acc)
            
            #logging all important metric
            logger.info(f"{'>>'*30} Score {'<<'*30}")
            logger.info(f"Train Score\t\t Test Score\t\t Average Score")
            logger.info(f"{train_acc}\t\t {test_acc}\t\t{model_accuracy}")

            logger.info(f"{'>>'*30} Loss {'<<'*30}")
            logger.info(f"Diff test train accuracy: [{diff_test_train_acc}].") 
            logger.info(f"Train root mean squared error: [{train_rmse}].")
            logger.info(f"Test root mean squared error: [{test_rmse}].")


            #if model accuracy is greater than base accuracy and train and test score is within certain thershold
            #we will accept that model as accepted model
            if model_accuracy >= base_accuracy and diff_test_train_acc < 0.05:
                base_accuracy = model_accuracy
                metric_info_artifact = MetricInfoArtifact(model_name=model_name,
                                                        model_object=model,
                                                        train_rmse=train_rmse,
                                                        test_rmse=test_rmse,
                                                        train_accuracy=train_acc,
                                                        test_accuracy=test_acc,
                                                        model_accuracy=model_accuracy,
                                                        index_number=index_number)

                logger.info(f"Acceptable model found {metric_info_artifact}. ")
            index_number += 1
        if metric_info_artifact is None:
            logger.info(f"No model found with higher accuracy than base accuracy")
        return metric_info_artifact
    except Exception as e:
        raise e


In [201]:
from cmath import log
import importlib
from pyexpat import model
import numpy as np
import yaml
from typing import List

In [202]:
try:
    config=ConfigurationManager()
    model_trainer=ModelTrainer(config=config)
    logger.info("training started ")
    model_trainer.get_trained_model()
    logger.info("training completed")
except Exception as e:
    raise e

[2023-03-30 14:25:15,499: INFO: common]: yaml file: config\config.yaml loaded successfully
[2023-03-30 14:25:15,501: INFO: common]: yaml file: params.yaml loaded successfully
[2023-03-30 14:25:15,503: INFO: common]: created directory at: artifacts
[2023-03-30 14:25:15,513: INFO: common]: yaml file: config\config.yaml loaded successfully
[2023-03-30 14:25:15,516: INFO: common]: yaml file: params.yaml loaded successfully
[2023-03-30 14:25:15,518: INFO: common]: created directory at: artifacts
[2023-03-30 14:25:15,520: INFO: 714097908]: training started 
[2023-03-30 14:25:15,522: INFO: 2388430155]: loading training and testing data sets
[2023-03-30 14:25:15,527: INFO: 2388430155]: train and test data generated for data training
[2023-03-30 14:25:15,529: INFO: 2388430155]: initializing model by using config filepath
[2023-03-30 14:25:15,535: INFO: 2388430155]: expected accuracy:0.6
[2023-03-30 14:25:15,536: INFO: 2157547986]: importing module:sklearn.linear_model with class:LinearRegressio

In [154]:
from sklearn.metrics import r2_score,mean_squared_error

In [7]:
from insurence_premium.components.data_model_trainer import ModelTrainer

c:\Users\somit\Downloads\project_ineuron\insurence_premium_prediction\env\lib\site-packages\evidently\analyzers\__init__.py:3: UserWarning: analyzers are deprecated, use metrics instead
  warnings.warn("analyzers are deprecated, use metrics instead")
c:\Users\somit\Downloads\project_ineuron\insurence_premium_prediction\env\lib\site-packages\evidently\model_profile\__init__.py:8: UserWarning: model profiles are deprecated, use metrics instead
  warnings.warn("model profiles are deprecated, use metrics instead")
c:\Users\somit\Downloads\project_ineuron\insurence_premium_prediction\env\lib\site-packages\evidently\dashboard\__init__.py:8: UserWarning: dashboards are deprecated, use metrics instead
  warnings.warn("dashboards are deprecated, use metrics instead")


from insure

In [8]:
from insurence_premium.entity.model_entity import GridSearchedBestModel

In [2]:
import insurence_premium

In [2]:
from insurence_premium.components.data_model_trainer import ModelTrainer

c:\Users\somit\Downloads\project_ineuron\insurence_premium_prediction\env\lib\site-packages\evidently\analyzers\__init__.py:3: UserWarning: analyzers are deprecated, use metrics instead
  warnings.warn("analyzers are deprecated, use metrics instead")
c:\Users\somit\Downloads\project_ineuron\insurence_premium_prediction\env\lib\site-packages\evidently\model_profile\__init__.py:8: UserWarning: model profiles are deprecated, use metrics instead
  warnings.warn("model profiles are deprecated, use metrics instead")
c:\Users\somit\Downloads\project_ineuron\insurence_premium_prediction\env\lib\site-packages\evidently\dashboard\__init__.py:8: UserWarning: dashboards are deprecated, use metrics instead
  warnings.warn("dashboards are deprecated, use metrics instead")


ModuleNotFoundError: No module named 'insurence_premium.entiy'

In [1]:
from insurence_premium.components.data_model_trainer import ModelTrainer

c:\Users\somit\Downloads\project_ineuron\insurence_premium_prediction\env\lib\site-packages\evidently\analyzers\__init__.py:3: UserWarning: analyzers are deprecated, use metrics instead
  warnings.warn("analyzers are deprecated, use metrics instead")
c:\Users\somit\Downloads\project_ineuron\insurence_premium_prediction\env\lib\site-packages\evidently\model_profile\__init__.py:8: UserWarning: model profiles are deprecated, use metrics instead
  warnings.warn("model profiles are deprecated, use metrics instead")
c:\Users\somit\Downloads\project_ineuron\insurence_premium_prediction\env\lib\site-packages\evidently\dashboard\__init__.py:8: UserWarning: dashboards are deprecated, use metrics instead
  warnings.warn("dashboards are deprecated, use metrics instead")


ModuleNotFoundError: No module named 'insurence_premium.entiy'

In [5]:
from insurence_premium.entity.model_entity import MetricInfoArtifact ,InitialisedModelDetail ,